### Flask 서버 구축

In [1]:
# import csv

# with open('X_train1.csv', 'r', encoding='utf-8-sig') as f: 
#   rdr = csv.reader(f) 
#   test = list(rdr)[0]

# test = list(test)
# for idx, i in enumerate(test):
#   test[idx] = i.strip('][').split(', ')

# for i in range(len(test)):
#   for j in range(len(test[i])):
#     test[i][j] = test[i][j][1:-1]

In [2]:
from flask import Flask # 플라스크 클래스 임포트
from flask import request, redirect, Request, Response
import pickle # 데이터 저장/로딩을 위한 모듈
import pandas as pd
import numpy as np
app = Flask(__name__) # 내장변수 name을 이용해 서버를 구동시키는 객체 생성

from tensorflow.keras.models import load_model
from werkzeug.utils import secure_filename
import re
import time
import sys
from pandas.core.frame import DataFrame
import json
import requests

#한글 지원 방법
import os

def detect():
  !python detect.py --weights "runs/train/light_yolov5s_results/weights/best.pt" --source "test/video_cross3_1.mp4" --save-txt --save-conf

def judge(num):
  exp_path = f"runs/detect/exp{num}/labels/"
  text_name = "video_cross3_1"

  total_txt_path = exp_path +text_name

  with open(f'{total_txt_path}.txt', 'r') as f:
      data = f.readlines()

  column_name = ['fram','class','x','y','w','h','p']
  data_split = [x.strip().split() for x in data[:]]

  df = pd.DataFrame(data_split, columns = column_name)
  df_crop=df.tail(30)

  red=df_crop.loc[df_crop['class']=='0',:]
  yellow=df_crop.loc[df_crop['class']=='1',:]
  green=df_crop.loc[df_crop['class']=='2',:]
  white_dotted=df_crop.loc[df_crop['class']=='3',:]
  white_solid=df_crop.loc[df_crop['class']=='4',:]
  cross=df_crop.loc[df_crop['class']=='5',:]
  straight=df_crop.loc[df_crop['class']=='6',:]
  right=df_crop.loc[df_crop['class']=='7',:]
  left=df_crop.loc[df_crop['class']=='8',:]

  #print('*****cross******')
  c=0
  for i in range(len(cross.index)):
    c+=float(cross.iloc[i]['p'])
  print('cross:',c)


  #print('*****straight******')
  s=0
  for i in range(len(straight.index)):
    s+=float(straight.iloc[i]['p'])
  print('straight:',s)


  # print('*****red******')
  r=0
  for i in range(len(red.index)):
    r+=float(red.iloc[i]['p'])
  print('red:', r)

  #print('*****yellow******')
  y=0
  for i in range(len(yellow.index)):
    y+=float(yellow.iloc[i]['p'])
  print('yellow:',y)

  #print('*****green******')
  g=0
  for i in range(len(green.index)):
    g+=float(green.iloc[i]['p'])
  print('green:',g)



  #print('*****right******')
  side_r=0
  for i in range(len(right.index)):
    side_r+=float(right.iloc[i]['p'])
  print('right:',side_r)

  #print('*****left******')
  side_l=0
  for i in range(len(left.index)):
    side_l+=float(left.iloc[i]['p'])
  print('left:',side_l)

  #차선검출 - 우측: 점선
  wd_r=0
  for i in range(len(white_dotted.index)):
    if float(white_dotted.iloc[i]['x'])<0.5:
      wd_r+=float(white_dotted.iloc[i]['p'])
  print('white_dotted(right):',wd_r)

  #차선검출 - 우측: 실선
  ws_r=0
  for i in range(len(white_solid.index)):
    if float(white_solid.iloc[i]['x'])<0.5:
      ws_r+=float(white_solid.iloc[i]['p'])
  print('white_solid(right):',ws_r)



  #차선검출 - 좌측: 점선

  wd_l=0
  for i in range(len(white_dotted.index)):
    if float(white_dotted.iloc[i]['x'])>=0.5:
      wd_l+=float(white_dotted.iloc[i]['p'])
  print('white_dotted(left):',wd_l)
        

  #차선검출 - 좌측: 실선
  ws_l=0
  for i in range(len(white_solid.index)):
    if float(white_solid.iloc[i]['x'])>=0.5:
      ws_l+=float(white_solid.iloc[i]['p'])
  print('white_solid(left):',ws_l)


  ############### 과실비율 판별 ###############
  print('*'*25)
  rate = 0
  #cross>straight
  if c>=s:
    print('도로유형 판별: 교차로' )
    if g>=r and g>=r : 
      rate = 0
      print('신호등 : 녹색진입' )
    elif y>=r and y>=g : 
      rate = 30
      print('신호등 : 황색진입' )
    elif r>=g and r>=y : 
      rate = 50
      print('신호등 : 적색진입' )
    else: rate =0

  else: 
    print('도로유형 판별: 직진차로' )

    if side_l>side_r:
      print('진입방향 : 좌측' )    
      if wd_l > ws_l :
        print('차선종류 : 흰색 점선 ' )
        rate = 30
      else :
        print('차선종류 : 흰색 실선 ' ) 
        rate = 10
    
    else: 
      print('진입방향 : 우측' ) # 미검출 또는 같을경우 우측으로 판단
      if wd_r > ws_r :
        print('차선종류 : 흰색 점선 ' )
        rate = 30
      else :
        print('차선종류 : 흰색 실선 ' ) 
        rate = 10

  print('최종 과실비율: ', rate)
  
  return rate

def message(tokken):
  url = "https://kapi.kakao.com/v2/api/talk/memo/default/send"

  # 사용자 토큰
  headers = {
      "Authorization": "Bearer " + tokken
  }

  template = {
      "object_type" : "list",
      "header_title" : "영상분석 완료",
      "header_link" : {
          "web_url" : "www.naver.com",
          "mobile_web_url" : "www.naver.com"
      },
      "contents" : [
          {
              "title" : "본인 0 : 상대방 100",
              "description" : "교차로 사고",
              "image_url" : "https://media.bunjang.co.kr/product/169940835_1_1636957290_w%7Bres%7D.jpg",
              "image_width" : 50, "image_height" : 50,
              "link" : {
                  "web_url" : "www.naver.com",
                  "mobile_web_url" : "www.naver.com"
              }
          },
          {
              "title" : "교차로(녹색진입)",
              "description" : "",
              "image_url" : "https://media.bunjang.co.kr/product/169940835_1_1636957290_w%7Bres%7D.jpg",
              "image_width" : 50, "image_height" : 50,
              "link" : {
                  "web_url" : "www.naver.com",
                  "mobile_web_url" : "www.naver.com"
              }
          }
          
      ],
      "buttons" : [
          {
              "title" : "웹으로 이동",
              "link" : {
                  "web_url" : "www.naver.com",
                  "mobile_web_url" : "www.naver.com"
              }
          }
      ]
      
  }

  data = {
      "template_object" : json.dumps(template)
  }

  response = requests.post(url, data=data, headers=headers)
  print(response.status_code)
  if response.json().get('result_code') == 0:
      print('메시지를 성공적으로 보냈습니다.')
  else:
      print('메시지를 성공적으로 보내지 못했습니다. 오류메시지 : ' + str(response.json()))


  


@app.route("/uploadFlask", methods=["GET",'POST'])
def predict0() :
    if request.method == 'POST':
      print("======= upload 시작 =======")
      f = request.files['file']
      #저장할 경로 + 파일명
      f.save("./test/" + secure_filename(f.filename))
      print('uploads 디렉토리 -> 파일 업로드 성공!')
      print("======= upload 끝 =======")

      print("======= detect 시작 =======")
      detect()
      print("======= detect 끝 =======")

      print("======= judge 시작 =======")
      result = judge(20)
      print("======= judge 끝 =======")

      print("======= message 시작 =======")
      message("6EuEDHtLWDfb0pX0ofRy0M3XeFpG6nZOh0XsqzlwCisMpwAAAYIia_2E")
      print("======= message 완료 =======")



      return f"<h1>uploads 디렉토리 -> 파일 업로드 성공!</h1><h1>과실비율 본인 : {result}  상대방 : {(100 - int(result))} </h1>"
    else :
        # 넘어온 값을 전처리
        print("Get 방식")
        return "Get 방식"


if __name__ == "__main__" : # .py 파일에서 main함수 역할
    app.run(host="localhost", port="5000")

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:5000/ (Press CTRL+C to quit)


======= upload 시작 =======
1단계
2단계
3단계
uploads 디렉토리 -> 파일 업로드 성공!
======= upload 끝 =======
======= detect 시작 =======


detect: weights=['runs/train/light_yolov5s_results/weights/best.pt'], source=test/video_cross3_1.mp4, data=data\coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5  2022-7-10 Python-3.8.13 torch-1.8.0+cu111 CUDA:0 (NVIDIA GeForce RTX 2070, 8192MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7034398 parameters, 0 gradients, 15.9 GFLOPs
video 1/1 (1/100) C:\Users\smhrd\Desktop\Flask(Model)\test\video_cross3_1.mp4: 384x640 1 straigt, Done. (0.015s)
video 1/1 (2/100) C:\Users\smhrd\Desktop\Flask(Model)\test\video_cross3_1.mp4: 384x640 1 straigt, Done. (0.012s)
video 1/1 (3/100) C:\Users\smhrd\Desktop\Flask(Model)\test\video_cross3_1.mp4: 384x640 1 straigt, Done. (0.0

======= detect 끝 =======
======= judge 시작 =======
cross: 14.000099
straight: 1.990434
red: 0
yellow: 0
green: 1.602109
right: 0
left: 0
white_dotted(right): 0
white_solid(right): 0
white_dotted(left): 0
white_solid(left): 0
*************************
도로유형 판별: 교차로
신호등 : 녹색진입
최종 과실비율:  0
======= judge 끝 =======
======= message 시작 =======
200
메시지를 성공적으로 보냈습니다.
======= message 완료 =======
